#Variational Quantum Eigensolver

Variational quantum eigensolver (VQE) is a hybrid quantum-classical algorithm that finds the smallest eigenvalue (and corresponding eigenvector) of a given Hamiltonian. One of the main applications of the algorithm is finding ground state energy of molecules. It has a big advantage over IQPE (iterative quantum phase estimation) and QPE (quantum phase estimation) algorithms, that also can be used for finding the ground state energy of a molecule. The main advantage is that VQE uses much smaller circuit depths (or gates) then IQPE and QPE, what is very important for NISQ (Noisy Intermediate-Scale Quantum) era quantum computation. In the NISQ era (now!) we are working with qubits that are very noisy because they are not isolated from the environment well enough. Thus, there is small and finite time to work with qubits until they will be "spoiled", because of the environment, imperfect gates and etc. This restriction gives a big advantage to those algorithms (like VQE) that are using small depth circuits.

##Question
Using VQE-like circuits, created by yourself from scratch, find the lowest eigenvalue of the following matrix:

```
1  0  0  0 
0  0 -1  0
0 -1  0  0 
0  0  0  1
```


In [ ]:
#!pip install qiskit

In [ ]:
from random import random
from scipy.optimize import minimize
import numpy as np
from numpy import kron
from qiskit import *
from qiskit.circuit.library.standard_gates import U2Gate
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.algorithms import NumPyEigensolver

##Decomposing the given matrix into the sum of its Pauli terms

This will find the coefficients of the pauli matrices. For the given matrix, the decomposition of the Hamiltonian will be as follows:
$$
H = 0.5I - 0.5X - 0.5Y + 0.5Z
$$

In [ ]:
def HS(M1, M2):
    return (np.dot(M1.conjugate().transpose(), M2)).trace()

def c2s(c):
    if c == 0.0:
        return "0"
    if c.imag == 0:
        return "%g" % c.real
    elif c.real == 0:
        return "%gj" % c.imag
    else:
        return "%g+%gj" % (c.real, c.imag)

def decompose(H):
    sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)
    sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
    sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
    id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
    S = [id, sx, sy, sz]
    labels = ['I', 'sigma_x', 'sigma_y', 'sigma_z']
    values = []
    for i in range(4):
        for j in range(4):
            label = labels[i] + ' \otimes ' + labels[j]
            a_ij = 0.25 * HS(kron(S[i], S[j]), H)
            values.append(c2s(a_ij))
    coef = []
    for value in values:
        if (float(value) != 0.0):
            coef.append(float(value))
    return coef


In [ ]:
H = np.array(np.diag([1,-1,-1,1]), dtype=np.complex128)
H[[1, 2]] = H[[2, 1]]
coefficients = decompose(H)
print('The coefficients of Pauli Matrices are: ', coefficients)

The coefficients of Pauli Matrices are:  [0.5, -0.5, -0.5, 0.5]


In [ ]:
def hamiltonian_operator(a, b, c, d):
    pauli_dict = {
        'paulis': [{"coeff": {"imag": 0.0, "real": a}, "label": "I"},
                   {"coeff": {"imag": 0.0, "real": b}, "label": "Z"},
                   {"coeff": {"imag": 0.0, "real": c}, "label": "X"},
                   {"coeff": {"imag": 0.0, "real": d}, "label": "Y"}
                   ]
    }
    return WeightedPauliOperator.from_dict(pauli_dict)

H = hamiltonian_operator(a=coefficients[0], b=coefficients[3], c=coefficients[1], d=coefficients[2])

exact_result = NumPyEigensolver(H).run()
reference_energy = min(np.real(exact_result.eigenvalues))
print('The lowest energy state is: ', reference_energy)

The lowest energy state is:  -0.3660254037844386


In [ ]:
H_gate = U2Gate(0, np.pi).to_matrix()
print("H_gate:")
print((H_gate * np.sqrt(2)).round(5))

Y_gate = U2Gate(0, np.pi/2).to_matrix()
print("Y_gate:")
print((Y_gate * np.sqrt(2)).round(5))

H_gate:
[[ 1.+0.j  1.-0.j]
 [ 1.+0.j -1.+0.j]]
Y_gate:
[[ 1.+0.j -0.-1.j]
 [ 1.+0.j  0.+1.j]]


In [ ]:
def quantum_state_preparation(circuit, parameters):
  q = circuit.qregs[0] # q is the quantum register where the info about qubits is stored
  circuit.rx(parameters[0], q[0]) # q[0] is our one and only qubit XD
  circuit.ry(parameters[1], q[0])
  return circuit

In [ ]:
def vqe_circuit(parameters, measure):
    """
    Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'Z' stands for Z measurement.
    :return: quantum circuit.
    """
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    circuit = QuantumCircuit(q, c)

    # quantum state preparation
    circuit = quantum_state_preparation(circuit, parameters)

    # measurement
    if measure == 'Z':
        circuit.measure(q[0], c[0])
    elif measure == 'X':
        circuit.u2(0, np.pi, q[0])
        circuit.measure(q[0], c[0])
    elif measure == 'Y':
        circuit.u2(0, np.pi/2, q[0])
        circuit.measure(q[0], c[0])
    else:
        raise ValueError('Not valid input for measurement: input should be "X" or "Y" or "Z"')

    return circuit

In [ ]:
def quantum_module(parameters, measure):
    # measure
    if measure == 'I':
        return 1
    elif measure == 'Z':
        circuit = vqe_circuit(parameters, 'Z')
    elif measure == 'X':
        circuit = vqe_circuit(parameters, 'X')
    elif measure == 'Y':
        circuit = vqe_circuit(parameters, 'Y')
    else:
        raise ValueError('Not valid input for measurement: input should be "I" or "X" or "Z" or "Y"')
    
    shots = 8192
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        sign = +1
        if measure_result == '1':
            sign = -1
        expectation_value += sign * counts[measure_result] / shots
        
    return expectation_value

In [ ]:
def pauli_operator_to_dict(pauli_operator):
    """
    from WeightedPauliOperator return a dict:
    {I: 0.7, X: 0.6, Z: 0.1, Y: 0.5}.
    :param pauli_operator: qiskit's WeightedPauliOperator
    :return: a dict in the desired form.
    """
    d = pauli_operator.to_dict()
    paulis = d['paulis']
    paulis_dict = {}

    for x in paulis:
        label = x['label']
        coeff = x['coeff']['real']
        paulis_dict[label] = coeff

    return paulis_dict
pauli_dict = pauli_operator_to_dict(H)

In [ ]:
def vqe(parameters):
        
    # quantum_modules
    quantum_module_I = pauli_dict['I'] * quantum_module(parameters, 'I')
    quantum_module_Z = pauli_dict['Z'] * quantum_module(parameters, 'Z')
    quantum_module_X = pauli_dict['X'] * quantum_module(parameters, 'X')
    quantum_module_Y = pauli_dict['Y'] * quantum_module(parameters, 'Y')
    
    # summing the measurement results
    classical_adder = quantum_module_I + quantum_module_Z + quantum_module_X + quantum_module_Y
    
    return classical_adder

In [ ]:
parameters_array = np.array([np.pi, np.pi])
tol = 1e-3 # tolerance for optimization precision.

vqe_result = minimize(vqe, parameters_array, method="Powell", tol=tol)
print('The exact ground state energy is: {}'.format(reference_energy))
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))

The exact ground state energy is: -0.3660254037844386
The estimated ground state energy from VQE algorithm is: -0.367431640625
